In [ ]:
import json
import uuid
from sqlalchemy import create_engine

from utils import reset_db, get_session, model_to_dict
from data.models import udahub

# Udahub Application

## Core Database

**Init DB**

In [ ]:
udahub_db = "data/core/udahub.db"

In [ ]:
reset_db(udahub_db)

In [ ]:
engine = create_engine(f"sqlite:///{udahub_db}", echo=False)
udahub.Base.metadata.create_all(bind=engine)

**Account**

In [ ]:
account_id = "cultpass"
account_name = "CultPass Card"

In [ ]:
with get_session(engine) as session:
    account = udahub.Account(
        account_id=account_id,
        account_name=account_name,
    )
    session.add(account)

## Integrations

**Knowledge Base**

In [ ]:
cultpass_articles = []

with open('data/external/cultpass_articles.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_articles.append(json.loads(line))

print(f"Loaded {len(cultpass_articles)} articles")

In [ ]:
assert len(cultpass_articles) >= 14, f"Expected at least 14 articles, got {len(cultpass_articles)}"
print(f"\u2705 Article count check passed: {len(cultpass_articles)} articles")

In [ ]:
with get_session(engine) as session:
    kb = []
    for article in cultpass_articles:
        knowledge = udahub.Knowledge(
            article_id=str(uuid.uuid4()),
            account_id=account_id,
            title=article["title"],
            content=article["content"],
            tags=article["tags"]
        )
        kb.append(knowledge)
    session.add_all(kb)
    print(f"\u2705 Loaded {len(kb)} articles into Knowledge table")

**Ticket**

In [ ]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [ ]:
ticket_info = {
    "status": "open",
    "content": "I can't log in to my Cultpass account.",
    "owner_id": cultpass_users[0]["id"],
    "owner_name": cultpass_users[0]["name"],
    "role": "user",
    "channel": "chat",
    "tags": "login, access",
}

In [ ]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()

    if not user:
        user = udahub.User(
            user_id=str(uuid.uuid4()),
            account_id=account_id,
            external_user_id=ticket_info["owner_id"],
            user_name=ticket_info["owner_name"],
        )
    
    ticket = udahub.Ticket(
        ticket_id=str(uuid.uuid4()),
        account_id=account_id,
        user_id=user.user_id,
        channel=ticket_info["channel"],
    )
    metadata = udahub.TicketMetadata(
        ticket_id=ticket.ticket_id,
        status=ticket_info["status"],
        main_issue_type=None,
        tags=ticket_info["tags"],
    )

    first_message = udahub.TicketMessage(
        message_id=str(uuid.uuid4()),
        ticket_id=ticket.ticket_id,
        role=ticket_info["role"],
        content=ticket_info["content"],
    )

    session.add_all([user, ticket, metadata, first_message])

## RAG Indexing

Embed all knowledge base articles into ChromaDB for semantic search.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from agentic.rag.indexer import index_knowledge_base

count = index_knowledge_base()
print(f"\u2705 Indexed {count} articles into ChromaDB")

In [ ]:
from agentic.rag.retriever import retriever

results = retriever.search("how to cancel subscription")
print(f"\nRAG test - 'how to cancel subscription':")
for r in results:
    print(f"  [{r['confidence']:.4f}] {r['title']}")

assert len(results) > 0, "RAG search returned no results"
assert results[0]['confidence'] > 0.5, f"Top result confidence too low: {results[0]['confidence']}"
print(f"\n\u2705 RAG pipeline verified")

# Tests

In [ ]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    print(account)

In [ ]:
with get_session(engine) as session:
    account = session.query(udahub.Account).filter_by(
        account_id=account_id
    ).first()
    articles = account.knowledge_articles
    for article in articles:
        print(article)
    assert len(articles) == 18, f"Expected 18 articles, got {len(articles)}"

In [ ]:
with get_session(engine) as session:
    users = session.query(udahub.User).all()
    for user in users:
        print(user)

In [ ]:
with get_session(engine) as session:
    user = session.query(udahub.User).filter_by(
        account_id=account_id,
        external_user_id=ticket_info["owner_id"],
    ).first()
    
    ticket = user.tickets[0]
    for message in ticket.messages:
        print(message)